In [1]:
import torch

### Why you need a good init

To understand why initialization is important in a neural net, we'll focus on the basic operation you have there: matrix multiplications. So let's just take a vector `x`, and a matrix `a` initialized randomly, then multiply them 100 times (as if we had 100 layers). 

In [9]:
x = torch.randn(512)
a = torch.randn(512,512)

In [11]:
x.mean(), x.std()

(tensor(-0.0103), tensor(0.9294))

In [12]:
for i in range(100): x = a @ x

In [13]:
x.mean(),x.std()

(tensor(nan), tensor(nan))

The problem you'll get with that is activation explosion: very soon, your activations will go to nan. We can even ask the loop to break when that first happens:

In [14]:
x = torch.randn(512)
a = torch.randn(512,512)

In [15]:
for i in range(100): 
    x = a @ x
    if x.std() != x.std(): break

In [16]:
i

28

In [27]:
x = torch.randn(512)
a = torch.randn(512,512)
for i in range(100): 
    x = a @ x
    print(i, x.std())
    if x.std() != x.std(): break

0 tensor(22.3095)
1 tensor(511.3192)
2 tensor(11501.5986)
3 tensor(264891.8125)
4 tensor(5965597.5000)
5 tensor(1.3041e+08)
6 tensor(3.0323e+09)
7 tensor(6.6385e+10)
8 tensor(1.5812e+12)
9 tensor(3.3721e+13)
10 tensor(7.7499e+14)
11 tensor(1.6723e+16)
12 tensor(3.7349e+17)
13 tensor(8.2272e+18)
14 tensor(1.8734e+20)
15 tensor(4.3132e+21)
16 tensor(1.0227e+23)
17 tensor(2.3789e+24)
18 tensor(5.4520e+25)
19 tensor(1.1570e+27)
20 tensor(2.5651e+28)
21 tensor(6.0166e+29)
22 tensor(1.4839e+31)
23 tensor(3.4380e+32)
24 tensor(7.9114e+33)
25 tensor(1.8063e+35)
26 tensor(4.1237e+36)
27 tensor(9.3939e+37)
28 tensor(nan)


It only takes around 30 multiplications! On the other hand, if you initialize your activations with a scale that is too low, then you'll get another problem:

In [28]:
x = torch.randn(512)
a = torch.randn(512,512) * 0.01

In [29]:
for i in range(100): x = a @ x

In [30]:
x.mean(),x.std()

(tensor(0.), tensor(0.))

In [31]:
x = torch.randn(512)
a = torch.randn(512,512) * 0.01
for i in range(100): 
    x = a @ x
    print(i, x.std())
    if x.std() ==0: break

0 tensor(0.2244)
1 tensor(0.0512)
2 tensor(0.0116)
3 tensor(0.0026)
4 tensor(0.0006)
5 tensor(0.0001)
6 tensor(2.9488e-05)
7 tensor(6.9368e-06)
8 tensor(1.4962e-06)
9 tensor(3.3060e-07)
10 tensor(8.0127e-08)
11 tensor(1.8077e-08)
12 tensor(4.0544e-09)
13 tensor(8.8686e-10)
14 tensor(1.8916e-10)
15 tensor(4.1768e-11)
16 tensor(9.4596e-12)
17 tensor(2.0430e-12)
18 tensor(4.4832e-13)
19 tensor(1.0471e-13)
20 tensor(2.4509e-14)
21 tensor(5.2914e-15)
22 tensor(1.1615e-15)
23 tensor(2.4594e-16)
24 tensor(5.5930e-17)
25 tensor(1.2067e-17)
26 tensor(2.6612e-18)
27 tensor(5.7656e-19)
28 tensor(1.2886e-19)
29 tensor(2.7758e-20)
30 tensor(6.0358e-21)
31 tensor(1.2946e-21)
32 tensor(3.0053e-22)
33 tensor(6.5882e-23)
34 tensor(1.4727e-23)
35 tensor(3.4777e-24)
36 tensor(7.8454e-25)
37 tensor(1.7530e-25)
38 tensor(4.1043e-26)
39 tensor(9.3277e-27)
40 tensor(2.1179e-27)
41 tensor(4.8878e-28)
42 tensor(1.0935e-28)
43 tensor(2.4739e-29)
44 tensor(5.3800e-30)
45 tensor(1.2400e-30)
46 tensor(2.8913e-31)


Here, every activation vanished to 0. So to avoid that problem, people have come with several strategies to initialize their weight matrices, such as:
- use a standard deviation that will make sure x and Ax have exactly the same scale
- use an orthogonal matrix to initialize the weight (orthogonal matrices have the special property that they preserve the L2 norm, so x and Ax would have the same sum of squares in that case)
- use [spectral normalization](https://arxiv.org/pdf/1802.05957.pdf) on the matrix A  (the spectral norm of A is the least possible number M such that `torch.norm(A@x) <= M*torch.norm(x)` so dividing A by this M insures you don't overflow. You can still vanish with this)

### The magic number for scaling

Here we will focus on the first one, which is the Xavier initialization. It tells us that we should use a scale equal to `1/math.sqrt(n_in)` where `n_in` is the number of inputs of our matrix.

In [32]:
import math

In [33]:
x = torch.randn(512)
a = torch.randn(512,512) / math.sqrt(512)

In [34]:
for i in range(100): x = a @ x

In [35]:
x.mean(),x.std()

(tensor(-0.1804), tensor(1.6565))

And indeed it works. Note that this magic number isn't very far from the 0.01 we had earlier.

In [36]:
1/ math.sqrt(512)

0.044194173824159216

But where does it come from? It's not that mysterious if you remember the definition of the matrix multiplication. When we do `y = a @ x`, the coefficients of `y` are defined by

$$y_{i} = a_{i,0} x_{0} + a_{i,1} x_{1} + \cdots + a_{i,n-1} x_{n-1} = \sum_{k=0}^{n-1} a_{i,k} x_{k}$$

or in code:
```
y[i] = sum([c*d for c,d in zip(a[i], x)])
```

Now at the very beginning, our `x` vector has a mean of roughly 0. and a standard deviation of roughly 1. (since we picked it that way).

In [37]:
x = torch.randn(512)
x.mean(), x.std()

(tensor(0.0967), tensor(1.0294))

NB: This is why it's extremely important to normalize your inputs in Deep Learning, the initialization rules have been designed with inputs that have a mean 0. and a standard deviation of 1.

If you need a refresher from your statistics course, the mean is the sum of all the elements divided by the number of elements (a basic average). The standard deviation shows whether the data points stay close to the mean or are far away from it. It's computed by the following formula:

$$\sigma = \sqrt{\frac{1}{n}\left[(x_{0}-m)^{2} + (x_{1}-m)^{2} + \cdots + (x_{n-1}-m)^{2}\right]}$$

where m is the mean and $\sigma$ (the greek letter sigma) is the standard deviation. To avoid that square root, we also often consider a quantity called the variance, which is $\sigma$ squared. 

Here we have a mean of 0, so the variance is just the mean of x squared, and the standard deviation is its square root.

If we go back to `y = a @ x` and assume that we chose weights for `a` that also have a mean of 0, we can compute the variance of `y` quite easily. Since it's random, and we may fall on bad numbers, we repeat the operation 100 times.

In [38]:
mean,sqr = 0.,0.
for i in range(100):
    x = torch.randn(512)
    a = torch.randn(512, 512)
    y = a @ x
    mean += y.mean().item()
    sqr  += y.pow(2).mean().item()
mean/100,sqr/100

(0.011828003078699112, 514.8822030639649)

Now that looks very close to the dimension of our matrix 512. And that's no coincidence! When you compute y, you sum 512 product of one element of a by one element of x. So what's the mean and the standard deviation of such a product of one element of `a` by one element of `x`? We can show mathematically that as long as the elements in `a` and the elements in `x` are independent, the mean is 0 and the std is 1.

This can also be seen experimentally:

In [39]:
mean,sqr = 0.,0.
for i in range(10000):
    x = torch.randn(1)
    a = torch.randn(1)
    y = a*x
    mean += y.item()
    sqr  += y.pow(2).item()
mean/10000,math.sqrt(sqr/10000)

(0.02564371645544379, 0.9938432075048306)

Then we sum 512 of those things that have a mean of zero, and a variance of 1, so we get something that has a mean of 0, and variance of 512. To go to the standard deviation, we have to add a square root, hence `math.sqrt(512)` being our magic number.

If we scale the weights of the matrix `a` and divide them by this `math.sqrt(512)`, it will give us a `y` of scale 1, and repeating the product as many times as we want and it won't overflow or vanish.

### Adding ReLU in the mix

We can reproduce the previous experiment with a ReLU, to see that this time, the mean shifts and the variance becomes 0.5. This time the magic number will be `math.sqrt(2/512)` to properly scale the weights of the matrix.

In [40]:
mean,sqr = 0.,0.
for i in range(10000):
    x = torch.randn(1)
    a = torch.randn(1)
    y = a*x
    y = 0 if y < 0 else y.item()
    mean += y
    sqr  += y ** 2
mean/10000,sqr/10000

(0.31586063899350064, 0.4944878516298298)

We can double check by running the experiment on the whole matrix product. The variance becomes 512/2 this time:

In [41]:
mean,sqr = 0.,0.
for i in range(100):
    x = torch.randn(512)
    a = torch.randn(512, 512)
    y = a @ x
    y = y.clamp(min=0)
    mean += y.mean().item()
    sqr  += y.pow(2).mean().item()
mean/100,sqr/100

(8.93101954460144, 252.5653419494629)

Or that scaling the coefficient with the magic number gives us a scale of 1.

In [42]:
mean,sqr = 0.,0.
for i in range(100):
    x = torch.randn(512)
    a = torch.randn(512, 512) * math.sqrt(2/512)
    y = a @ x
    y = y.clamp(min=0)
    mean += y.mean().item()
    sqr  += y.pow(2).mean().item()
mean/100,math.sqrt(sqr/100)

(0.5710994786024094, 1.0092722273550656)

The math behind is a tiny bit more complex, and you can find everything in the [Kaiming](https://arxiv.org/abs/1502.01852) and the [Xavier](http://proceedings.mlr.press/v9/glorot10a.html) paper but this gives the intuition behind those results.